In [1]:
import numpy as np
import warnings 
import sys
import pandas as pd
import scipy
from scipy.io import arff
if not sys.warnoptions:
    warnings.simplefilter("ignore")
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data, meta = scipy.io.arff.loadarff('mtr-datasets/scm1d.arff')
data = pd.DataFrame(data)
data.head()

,timeunit,storageCost,interestRate,compidx0lt2,compidx0lt2l1,compidx0lt2l2,compidx0lt2l4,compidx0lt2l8,compidx1lt2,compidx2lt2,...,MTLp7,MTLp8,MTLp9,MTLp10,MTLp11,MTLp12,MTLp13,MTLp14,MTLp15,MTLp16
0,9.0,27.0,8.0,881.606,882.253,826.727,810.837,848.833,1468.412,815.832,...,2292.0,2181.0,1619.0,1849.0,1707.0,1932.0,2052.0,2409.0,2279.0,2138.0
1,10.0,27.0,8.0,869.455,881.606,882.253,802.480,770.544,1407.148,828.687,...,2276.0,2231.0,1649.0,1813.0,1696.0,1988.0,2140.0,2366.0,2307.0,2154.0
2,11.0,27.0,8.0,850.916,869.455,881.606,826.727,784.625,1424.312,837.975,...,2264.0,2203.0,1651.0,1739.0,1677.0,1991.0,2087.0,2295.0,2185.0,2131.0
3,12.0,27.0,8.0,862.066,850.916,869.455,882.253,792.403,1411.209,803.296,...,2279.0,2203.0,1627.0,1747.0,1669.0,1885.0,2072.0,2292.0,2187.0,2170.0
4,13.0,27.0,8.0,842.781,862.066,850.916,881.606,810.837,1428.685,813.118,...,2241.0,2198.0,1626.0,1740.0,1663.0,1874.0,2074.0,2257.0,2136.0,2164.0


In [3]:
X = data.iloc[:,:-16].values
y = data.iloc[:,-16:].values

scaler = StandardScaler()

X = scaler.fit_transform(X)
y = scaler.fit_transform(y)

print(X.shape,y.shape)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

(9803, 280) (9803, 16)


In [4]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(7842, 280) (1961, 280) (7842, 16) (1961, 16)


In [5]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,f1_score,r2_score
def evaluate(y_train,y_test):
    print('MAE - ',mean_absolute_error(y_train,y_test))
    print('RMSE - ',mean_squared_error(y_train,y_test)**0.5)
    print('R-squared - ',r2_score(y_train,y_test))
    
    r2 = r2_score(y_train,y_test)
    
    n = 9803
    p = 280

    adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    print('Adjusted R-squared - ',adj_r2)

### KNN Regression

In [6]:
from sklearn.neighbors import KNeighborsRegressor

In [7]:
for i in range(1,20):
    model = KNeighborsRegressor(n_neighbors=i)
    model.fit(X_train,y_train)
    print(i,":",model.score(X_test,y_test))

1 : 0.8823048464990688
2 : 0.9078011153281513
3 : 0.9098496308636628
4 : 0.910501279934302
5 : 0.9084924315158178
6 : 0.9037659959383251
7 : 0.902545599223907
8 : 0.9005584898994894
9 : 0.8976691910638097
10 : 0.8945233303346466
11 : 0.8929788019836837
12 : 0.888816414785812
13 : 0.8845113587972739
14 : 0.8821306624495093
15 : 0.8793971138758923
16 : 0.8769755847164431
17 : 0.8749276742911527
18 : 0.8722948105920031
19 : 0.8693754989788555


In [8]:
model = KNeighborsRegressor(n_neighbors=9)
model.fit(X_train,y_train)
evaluate(scaler.inverse_transform(y_test),scaler.inverse_transform(model.predict(X_test)))

MAE -  50.06207150546774
RMSE -  87.67382199860869
R-squared -  0.8976691910638097
Adjusted R-squared -  0.8946600935525586


### Random Forest

In [9]:
from sklearn.ensemble import RandomForestRegressor

In [10]:
model = RandomForestRegressor(n_estimators=5)
model.fit(X_train,y_train)
evaluate(scaler.inverse_transform(y_test),scaler.inverse_transform(model.predict(X_test)))

MAE -  61.139743753187176
RMSE -  96.93371497143913
R-squared -  0.8750891384831487
Adjusted R-squared -  0.8714160612698828


In [11]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [12]:
model = Sequential()
model.add(Dense(128,activation='relu',input_dim=X_train.shape[1]))
model.add(Dense(64,activation='relu'))
model.add(Dense(16,activation='linear'))
model.compile(optimizer="adam",loss='mse')

In [13]:
model.fit(X_train,y_train,epochs=500,batch_size=50)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Epoch 1/500
7842/7842 [==============================] - 1s - loss: 0.3411     
Epoch 2/500
7842/7842 [==============================] - 0s - loss: 0.1734     
Epoch 3/500
7842/7842 [==============================] - 0s - loss: 0.1464     
Epoch 4/500
7842/7842 [==============================] - 0s - loss: 0.1334     
Epoch 5/500
7842/7842 [==============================] - 0s - loss: 0.1266     
Epoch 6/500
7842/7842 [==============================] - 0s - loss: 0.1180     
Epoch 7/500
7842/7842 [==============================] - 0s - loss: 0.1119     - ETA: 0s -
Epoch 8/500
7842/7842 [==============================] - 0s - loss: 0.1077     
Epoch 9/500
7842/7842 [==============================] - 0s - loss: 0.1043     
Epoch 10/500
7842/7842 [==============================] - 0s - loss: 0.1010     
Epoch 11/500
7842/7842 [==============================] - 0s - loss:

7842/7842 [==============================] - 0s - loss: 0.0527     
Epoch 93/500
7842/7842 [==============================] - 0s - loss: 0.0526     - ETA: 0s - 
Epoch 94/500
7842/7842 [==============================] - 0s - loss: 0.0538     
Epoch 95/500
7842/7842 [==============================] - 0s - loss: 0.0526     
Epoch 96/500
7842/7842 [==============================] - 0s - loss: 0.0522     
Epoch 97/500
7842/7842 [==============================] - 0s - loss: 0.0518     
Epoch 98/500
7842/7842 [==============================] - 0s - loss: 0.0527     
Epoch 99/500
7842/7842 [==============================] - 0s - loss: 0.0523     
Epoch 100/500
7842/7842 [==============================] - 0s - loss: 0.0514     
Epoch 101/500
7842/7842 [==============================] - 0s - loss: 0.0510     
Epoch 102/500
7842/7842 [==============================] - 0s - loss: 0.0512     
Epoch 103/500
7842/7842 [==============================] - 0s - loss: 0.0509     
Epoch 104/500
7842/7842 [

7842/7842 [==============================] - 0s - loss: 0.0421     
Epoch 193/500
7842/7842 [==============================] - 0s - loss: 0.0425     
Epoch 194/500
7842/7842 [==============================] - 0s - loss: 0.0423     
Epoch 195/500
7842/7842 [==============================] - 0s - loss: 0.0423     
Epoch 196/500
7842/7842 [==============================] - 0s - loss: 0.0411     
Epoch 197/500
7842/7842 [==============================] - 0s - loss: 0.0420     
Epoch 198/500
7842/7842 [==============================] - 0s - loss: 0.0418     
Epoch 199/500
7842/7842 [==============================] - 0s - loss: 0.0421     
Epoch 200/500
7842/7842 [==============================] - 0s - loss: 0.0429     
Epoch 201/500
7842/7842 [==============================] - 0s - loss: 0.0434     
Epoch 202/500
7842/7842 [==============================] - 0s - loss: 0.0426     
Epoch 203/500
7842/7842 [==============================] - 0s - loss: 0.0420     
Epoch 204/500
7842/7842 [=====

7842/7842 [==============================] - 0s - loss: 0.0383     
Epoch 292/500
7842/7842 [==============================] - 0s - loss: 0.0380     
Epoch 293/500
7842/7842 [==============================] - 0s - loss: 0.0380     
Epoch 294/500
7842/7842 [==============================] - 0s - loss: 0.0386     
Epoch 295/500
7842/7842 [==============================] - 0s - loss: 0.0386     
Epoch 296/500
7842/7842 [==============================] - 0s - loss: 0.0380     
Epoch 297/500
7842/7842 [==============================] - 0s - loss: 0.0382     
Epoch 298/500
7842/7842 [==============================] - 0s - loss: 0.0378     
Epoch 299/500
7842/7842 [==============================] - 0s - loss: 0.0377     
Epoch 300/500
7842/7842 [==============================] - 0s - loss: 0.0384     
Epoch 301/500
7842/7842 [==============================] - 0s - loss: 0.0377     
Epoch 302/500
7842/7842 [==============================] - 0s - loss: 0.0390     
Epoch 303/500
7842/7842 [=====

7842/7842 [==============================] - 0s - loss: 0.0360     
Epoch 391/500
7842/7842 [==============================] - 0s - loss: 0.0353     
Epoch 392/500
7842/7842 [==============================] - 0s - loss: 0.0365     
Epoch 393/500
7842/7842 [==============================] - 0s - loss: 0.0356     
Epoch 394/500
7842/7842 [==============================] - 0s - loss: 0.0354     
Epoch 395/500
7842/7842 [==============================] - 0s - loss: 0.0358     
Epoch 396/500
7842/7842 [==============================] - 0s - loss: 0.0351     
Epoch 397/500
7842/7842 [==============================] - 0s - loss: 0.0354     
Epoch 398/500
7842/7842 [==============================] - 0s - loss: 0.0358     
Epoch 399/500
7842/7842 [==============================] - 0s - loss: 0.0357     
Epoch 400/500
7842/7842 [==============================] - 0s - loss: 0.0351     
Epoch 401/500
7842/7842 [==============================] - 0s - loss: 0.0353     
Epoch 402/500
7842/7842 [=====

7842/7842 [==============================] - 0s - loss: 0.0340     
Epoch 490/500
7842/7842 [==============================] - 0s - loss: 0.0335     
Epoch 491/500
7842/7842 [==============================] - 0s - loss: 0.0332     
Epoch 492/500
7842/7842 [==============================] - 0s - loss: 0.0339     
Epoch 493/500
7842/7842 [==============================] - 0s - loss: 0.0337     
Epoch 494/500
7842/7842 [==============================] - 0s - loss: 0.0331     
Epoch 495/500
7842/7842 [==============================] - 0s - loss: 0.0331     
Epoch 496/500
7842/7842 [==============================] - 0s - loss: 0.0336     
Epoch 497/500
7842/7842 [==============================] - 0s - loss: 0.0330     
Epoch 498/500
7842/7842 [==============================] - 0s - loss: 0.0337     
Epoch 499/500
7842/7842 [==============================] - 0s - loss: 0.0338     
Epoch 500/500
7842/7842 [==============================] - 0s - loss: 0.0343     - ETA: 0s


In [14]:
evaluate(scaler.inverse_transform(y_test),scaler.inverse_transform(model.predict(X_test)))

MAE -  60.51870096976024
RMSE -  94.40328849680688
R-squared -  0.8819046917327311
Adjusted R-squared -  0.8784320298639183
